In [46]:
import argparse
import os
import sys
import torch
import torch.nn as nn
import cv2
import datasets
import models.resnet as ResNet
import models.senet as SENet
from extractor import Extractor
import utils
from torchvision import transforms
from PIL import Image
import pickle
from datasets.vgg_face2 import VGG_Faces2
import numpy as np
#img = cv2.imread("Gennaro_Gattuso.jpg")
#img = transforms.ToTensor()
model = SENet.senet50(num_classes = 8631,include_top = False)


In [38]:
utils.load_state_dict(model,'senet50_scratch_weight.pkl')
model.eval()


SENet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)

In [48]:
import torchvision

In [61]:
from keras import backend as K
from keras.preprocessing import image

In [58]:
def preprocess_input(x, data_format=None, version=1):
    x_temp = np.copy(x)
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if version == 1:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

    elif version == 2:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
    else:
        raise NotImplementedError

    return x_temp

In [64]:
img = Image.open("Andrea_Pirlo_1.jpg")
x = image.image_utils.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x, version=2) # or version=2
preds = model(x)
print('Predicted:', utils.decode_predictions(preds))

TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)


In [53]:
img = Image.open("Andrea_Pirlo_1.jpg")
img = torchvision.transforms.Resize(256)(img)
img = np.array(img, dtype=np.uint8)
mean_bgr = np.array([91.4953, 103.8827, 131.0912]) 
img = img[:, :, ::-1]  # RGB -> BGR
img = img.astype(np.float32)
img -= mean_bgr
img = img.transpose(2, 0, 1)  # C x H x W
img = torch.from_numpy(img).float()
#img = img.resize((256,256), Image.ANTIALIAS)
#img = transforms.ToTensor()(img)
img = img.unsqueeze(0)
output = model(img)
print(img.size())
print(output.size())
max = torch.argmax(output)
print(max)


RuntimeError: Given input size: (256x77x64). Calculated output size: (256x1x0). Output size is too small